In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
%autosave 5

Autosaving every 5 seconds


In [2]:
def loadData(dataset,split=0):
    if(split == 0):
        print("In non split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51)
    else:
        print("In split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51,
                                                            validation_split=split,subset='both')

In [3]:
def f1_score(y_true, y_pred): #taken from old keras source code 
    true_positives = K.sum(y_true*y_pred)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [4]:
def createModel(convFilters1, convFilters2, convFilters3, convFilters4, numberOfFCLayers, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate,L2Rate):
    model = keras.Sequential()
        
    #convPool1
    model.add(keras.layers.Conv2D(convFilters1,(3,3), activation='relu',padding='valid'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid'))
    #convPool2
    model.add(keras.layers.Conv2D(convFilters2,(3,3), activation='relu',padding='valid'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid'))
    #convPool3
    model.add(keras.layers.Conv2D(convFilters3,(3,3), activation='relu',padding='valid'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid'))
    #finalConv
    model.add(keras.layers.Conv2D(convFilters4,(3,3), activation='relu',padding='valid'))
    
    model.add(keras.layers.Flatten())
    
    for layer in range(numberOfFCLayers):
        if layer == numberOfFCLayers - 1:
            model.add(keras.layers.Dense(1,activation='sigmoid'))
        else:
            model.add(keras.layers.Dense(numberOfNeuronsPerFCLayer,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(L2Rate)))
            model.add(keras.layers.Dropout(dropoutRate))
            
    adamOptimizer = keras.optimizers.legacy.Adam(learning_rate=adamLearningRate)       
    model.compile(optimizer="adam",loss='binary_crossentropy', metrics=f1_score)
    return model

In [5]:
def createModelParametersDF(n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,
                            n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,
                            dropoutRates,adamLearningRates,L2Rates,trainScores,devScores):
    modelParameters = dict()
    modelParameters['n_convFilters1'] = n_convFilters1
    modelParameters['n_convFilters2'] = n_convFilters2
    modelParameters['n_convFilters3'] = n_convFilters3
    modelParameters['n_convFilters4'] = n_convFilters4
    modelParameters['n_FCLayers'] = n_FCLayers
    modelParameters['n_NeuronsPerFCLayers'] = n_NeuronsPerFCLayers
    modelParameters['n_Epochs'] = n_Epochs
    modelParameters['dropoutRate'] = dropoutRates
    modelParameters['adamLearningRates'] = adamLearningRates
    modelParameters['L2Rates'] = L2Rates
    modelParameters['trainScore'] = trainScores
    modelParameters['devScore'] = devScores

    modelParametersDF = pd.DataFrame(modelParameters, columns=modelParameters.keys())
    return modelParametersDF

In [6]:
def createRangeFromMidpoint(midpoint,range,mandatoryMinimum=1):
    possibleMin = int(midpoint-(range/2))
    possibleMin = max([mandatoryMinimum,possibleMin])
    possibleMax = int(midpoint+(range/2))
    possibleRange = np.arange(possibleMin,possibleMax)
    return possibleRange

In [7]:
def f1_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    
    tp = K.sum(y_true * y_pred)
    fp = K.sum((1 - y_true) * y_pred)
    fn = K.sum(y_true * (1 - y_pred))

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    loss = 1 - f1
    
    return loss

In [8]:
def generateDropoutRate(minVal=0,maxVal=1):
    dropoutRate = np.random.random() * (maxVal - minVal) + minVal
    dropoutRate = np.max([dropoutRate,0])
    dropoutRate = np.min([dropoutRate,0.999])
    return dropoutRate

In [9]:
def generateAdamLearningRate(minVal=1e-4,maxVal=1e-2):
    minVal = np.log10(minVal)
    maxVal = np.log10(maxVal)
    learningRatePower = np.random.random() * (maxVal - minVal) + minVal
    learningRate = np.power(10,learningRatePower)
    return learningRate

In [10]:
def generateL2(minVal=1e-2,maxVal=1e3):
    minVal = np.log10(minVal)
    maxVal = np.log10(maxVal)
    l2Power = np.random.random() * (maxVal - minVal) + minVal
    l2 = np.power(10,l2Power)
    return l2

In [11]:
def calculateCriticalPoints(top5ParamList):
    lowPoint = np.min(top5ParamList)
    highPoint = np.max(top5ParamList)
    return (lowPoint,highPoint)

In [12]:
def calculateLogisticCriticalPoints(top5ParamList):
    top5Log10ParamList = np.log10(top5ParamList)
    log10criticalPointTuple = calculateCriticalPoints(top5Log10ParamList)
    criticalPointTuple = (np.power(10,log10criticalPointTuple[0]),np.power(10,log10criticalPointTuple[1]))
    return criticalPointTuple

In [13]:
def getAdjustedRange(top5ParamList):
    lowerValue = int(np.max([1,np.min(top5ParamList)]))
    upperValue = int(np.max(top5ParamList))
    
    if lowerValue == upperValue:
        return createRangeFromMidpoint(lowerValue,2*lowerValue)
    return np.arange(lowerValue,upperValue)

In [14]:
def main():

    possibleConvFilters1 = createRangeFromMidpoint(32,64)
    possibleConvFilters2 = createRangeFromMidpoint(32,64)
    possibleConvFilters3 = createRangeFromMidpoint(32,64)
    possibleConvFilters4 = createRangeFromMidpoint(32,64)

    possibleNumberOfFCLayers = createRangeFromMidpoint(10,20)
    possibleNumberOfNeuronsPerFCLayer = createRangeFromMidpoint(10,20)

    possibleNumberOfEpochs = createRangeFromMidpoint(10,20)
    dropoutCriticalPoints = (0,1)
    adamLearningRateCriticalPoints = (1e-4,1e-2)
    L2CriticalPoints = (1e-2,1e3)    
    
    trial = 0
    
    train = loadData("training")
    dev,test = loadData("testing",.5)

    choose = np.random.choice

    n_convFilters1 = []
    n_convFilters2 = []
    n_convFilters3 = []
    n_convFilters4 = []
    n_FCLayers = []
    n_NeuronsPerFCLayers = []
    n_Epochs = []
    dropoutRates = []
    adamLearningRates = []
    L2Rates = []
    trainScores = []
    devScores = []
    
    while trial < 100:
        convFilters1 = choose(possibleConvFilters1)
        convFilters2 = choose(possibleConvFilters2)
        convFilters3 = choose(possibleConvFilters3)
        convFilters4 = choose(possibleConvFilters4)

        numberOfFCLayers = choose(possibleNumberOfFCLayers)
        numberOfNeuronsPerFCLayer = choose(possibleNumberOfNeuronsPerFCLayer)

        numberOfEpochs = choose(possibleNumberOfEpochs)
        
        dropoutRate = generateDropoutRate(dropoutCriticalPoints[0],dropoutCriticalPoints[1])
        adamLearningRate = generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
        L2Rate = generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
        

        model = createModel(convFilters1, convFilters2, convFilters3, convFilters4,
                            numberOfFCLayers, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate,L2Rate)
    
        model.fit(train,epochs=numberOfEpochs,verbose=0)

        model_path = f'../Models/FlipDetectionModelTrials/fd_model_{trial}.h5'
        model.save(model_path)
        model_size = os.path.getsize(model_path) / (1024 * 1024)
        if model_size < 40:
            print()
            print('trainScore')
            trainScore = model.evaluate(train)[1]
            print('devScore')
            devScore = model.evaluate(dev)[1]

            if devScore > 0.91:
                model_path = f'../Models/fd_model_{trial}.h5'
                model.save(model_path)

            n_convFilters1.append(convFilters1)
            n_convFilters2.append(convFilters2)
            n_convFilters3.append(convFilters3)
            n_convFilters4.append(convFilters4)
            
            n_FCLayers.append(numberOfFCLayers)
            n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
            n_Epochs.append(numberOfEpochs)
            
            adamLearningRates.append(adamLearningRate)
            dropoutRates.append(dropoutRate)
            L2Rates.append(L2Rate)
            trainScores.append(trainScore)
            devScores.append(devScore)
            
            print('concluding trial ',trial)
            trial += 1
        else:
            print(f'redoing trial {trial}. Model was {model_size}MB.')
            failedTrial = createModelParametersDF([convFilters1],[convFilters2],[convFilters3],[convFilters4],
                                                  [numberOfFCLayers],[numberOfNeuronsPerFCLayer],[numberOfEpochs],
                                                  [dropoutRate],[adamLearningRate],[L2Rate],[np.nan],[np.nan])
            display(failedTrial)
        
            
        if (trial % 10 == 9): 
            modelParametersDF = createModelParametersDF(n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,
                                                    n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,
                                                    dropoutRates,adamLearningRates,L2Rates,trainScores,devScores)
            modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
            display(modelParametersDF)
            
            top5 = modelParametersDF[0:5]
            possibleConvFilters1 = getAdjustedRange(top5['n_convFilters1'])
            possibleConvFilters2 = getAdjustedRange(top5['n_convFilters2'])
            possibleConvFilters3 = getAdjustedRange(top5['n_convFilters3'])
            possibleConvFilters4 = getAdjustedRange(top5['n_convFilters4'])
        
            possibleNumberOfFCLayers = getAdjustedRange(top5['n_FCLayers'])
            possibleNumberOfNeuronsPerFCLayer = getAdjustedRange(top5['n_NeuronsPerFCLayers'])
        
            possibleNumberOfEpochs = getAdjustedRange(top5['n_Epochs'])
            dropoutCriticalPoints = calculateCriticalPoints(top5['dropoutRate'])
            adamLearningRateCriticalPoints = calculateLogisticCriticalPoints(top5['adamLearningRates'])
            L2CriticalPoints = calculateLogisticCriticalPoints(top5['L2Rates'])

            n_convFilters1 = []
            n_convFilters2 = []
            n_convFilters3 = []
            n_convFilters4 = []
            n_FCLayers = []
            n_NeuronsPerFCLayers = []
            n_Epochs = []
            dropoutRates = []
            adamLearningRates = []
            L2Rates = []
            trainScores = []
            devScores = []
            
    

In [15]:
main()

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-07-13 03:08:41.392005: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-07-13 03:08:41.392141: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2392]
	 [[{{node Placeholder/_0}}]]
2024-07-13 03:08:41.733796: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz



trainScore
75/75 [==============================] - 16s 203ms/step - loss: 34.9210 - f1_score: 0.9689
devScore


2024-07-13 03:12:44.436927: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]
2024-07-13 03:12:44.437100: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 2s 135ms/step - loss: 34.9198 - f1_score: 0.9693
concluding trial  0

trainScore
75/75 [==============================] - 18s 207ms/step - loss: 39.9912 - f1_score: 0.9983
devScore
10/10 [==============================] - 2s 148ms/step - loss: 39.9911 - f1_score: 0.9983
concluding trial  1

trainScore
75/75 [==============================] - 12s 152ms/step - loss: 1.5862 - f1_score: 0.9674
devScore
10/10 [==============================] - 2s 106ms/step - loss: 1.5850 - f1_score: 0.9674
concluding trial  2

trainScore
75/75 [==============================] - 7s 88ms/step - loss: 384.3730 - f1_score: 0.9755
devScore
10/10 [==============================] - 1s 48ms/step - loss: 384.3721 - f1_score: 0.9755
concluding trial  3

trainScore
75/75 [==============================] - 8s 106ms/step - loss: 7.5578 - f1_score: 0.9716
devScore
10/10 [==============================] - 1s 66ms/step - loss: 7.5568 - f1_score: 0.9716
concluding trial  4

trainSco

...


trainScore
75/75 [==============================] - 9s 109ms/step - loss: 460.0095 - f1_score: 0.9953
devScore
10/10 [==============================] - 1s 71ms/step - loss: 460.0092 - f1_score: 0.9953
concluding trial  79

trainScore
75/75 [==============================] - 10s 126ms/step - loss: 452.9048 - f1_score: 0.9956
devScore
10/10 [==============================] - 1s 76ms/step - loss: 452.9046 - f1_score: 0.9956
concluding trial  80

trainScore
75/75 [==============================] - 9s 111ms/step - loss: 175.5267 - f1_score: 0.9969
devScore
10/10 [==============================] - 1s 71ms/step - loss: 175.5266 - f1_score: 0.9969
concluding trial  81

trainScore
75/75 [==============================] - 9s 113ms/step - loss: 121.4661 - f1_score: 0.9985
devScore
10/10 [==============================] - 1s 76ms/step - loss: 121.4660 - f1_score: 0.9985
concluding trial  82

trainScore
75/75 [==============================] - 9s 112ms/step - loss: 764.7535 - f1_score: 0.9916
devS

,n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
3,17,27,44,26,10,1,1,0.383519,0.004164,23.818091,0.998505,0.998505
6,17,27,25,26,19,1,1,0.380308,0.004150,23.717778,0.997834,0.997835
5,17,27,36,26,11,2,1,0.377107,0.004317,18.733253,0.997409,0.997408
8,17,27,50,26,17,2,1,0.383204,0.004230,23.973290,0.997122,0.997122
2,17,27,27,26,14,1,1,0.382704,0.004310,23.373158,0.996852,0.996853
7,17,27,34,26,18,4,1,0.380810,0.004203,22.516186,0.996463,0.996464
9,17,27,46,26,6,1,1,0.386877,0.004230,21.774689,0.996037,0.996037
1,17,27,39,26,13,2,1,0.377887,0.004158,19.086157,0.995649,0.995649
0,17,27,21,26,10,3,1,0.388368,0.004313,22.971442,0.995255,0.995256
4,17,27,26,26,17,3,1,0.387385,0.004196,23.150125,0.991609,0.991610



trainScore
75/75 [==============================] - 10s 123ms/step - loss: 170.1626 - f1_score: 0.9976
devScore
10/10 [==============================] - 1s 82ms/step - loss: 170.1626 - f1_score: 0.9976
concluding trial  89

trainScore
75/75 [==============================] - 8s 105ms/step - loss: 176.0776 - f1_score: 0.9989
devScore
10/10 [==============================] - 1s 64ms/step - loss: 176.0775 - f1_score: 0.9989
concluding trial  90

trainScore
75/75 [==============================] - 10s 122ms/step - loss: 193.9640 - f1_score: 0.9977
devScore
10/10 [==============================] - 1s 77ms/step - loss: 193.9639 - f1_score: 0.9977
concluding trial  91

trainScore
75/75 [==============================] - 9s 118ms/step - loss: 139.0609 - f1_score: 0.9974
devScore
10/10 [==============================] - 1s 74ms/step - loss: 139.0608 - f1_score: 0.9974
concluding trial  92

trainScore
75/75 [==============================] - 7s 83ms/step - loss: 67.6940 - f1_score: 0.9975
devSc

,n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
1,12,49,41,9,15,1,1,0.378933,0.004279,19.886132,0.998863,0.998863
9,28,23,44,7,18,1,1,0.382383,0.004163,19.430342,0.998238,0.998238
2,19,28,46,28,11,1,1,0.380742,0.004291,21.275000,0.997669,0.997669
0,14,51,39,38,14,1,1,0.380860,0.004233,22.258023,0.997627,0.997626
4,8,19,25,42,11,1,1,0.378228,0.004212,20.110269,0.997521,0.997521
3,25,13,41,6,10,1,1,0.377717,0.004285,19.074947,0.997353,0.997353
6,29,43,45,47,17,1,1,0.379566,0.004316,22.887958,0.996777,0.996777
7,21,32,28,17,16,1,1,0.379869,0.004181,18.936002,0.996741,0.996740
8,16,36,38,42,15,1,1,0.378918,0.004158,22.389972,0.996646,0.996647
5,5,17,30,21,13,1,1,0.381153,0.004229,18.778916,0.995643,0.995642



trainScore
75/75 [==============================] - 10s 121ms/step - loss: 218.3711 - f1_score: 0.9962
devScore
10/10 [==============================] - 1s 77ms/step - loss: 218.3709 - f1_score: 0.9962
concluding trial  99
